In [ ]:

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
#!/bin/bash
! kaggle datasets download kurianbenoy/20-newsgroup-dataset

Dataset URL: https://www.kaggle.com/datasets/kurianbenoy/20-newsgroup-dataset
License(s): unknown
20-newsgroup-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/20-newsgroup-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,label,texts
0,7,I wondering anyone could enlighten car I saw d...
1,4,A fair number brave souls upgraded SI clock os...
2,4,well folks mac plus finally gave ghost weekend...
3,1,Do Weitek address phone number I like get info...
4,14,From article C owCB n p world std com tombaker...


In [ ]:
df.shape

(11314, 2)

In [ ]:
df.isnull().sum()

,0
label,0
texts,312


In [ ]:
df.dropna(inplace = True)

In [ ]:
df.shape

(11002, 2)

In [ ]:
df['label'].value_counts().sort_index()

,count
label,
0,468
1,569
2,568
3,578
4,556
5,590
6,577
7,562
8,583


In [ ]:
import nltk
import string
import re
! pip install contractions

import contractions

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
stop_word = stopwords.words('english')

lemma = WordNetLemmatizer()

In [ ]:
def clean_text(text):
  text = contractions.fix(text)
  text = text.lower()
  text = re.sub(r'http\S+' , '' , text)
  text = re.sub(r'<.*?>' , '' , text)
  text = re.sub(r'[^a-z\s]', '', text)
  token = word_tokenize(text)
  final = [lemma.lemmatize(word) for word in token if word not in stop_word]
  return ' '.join(final)

In [ ]:
df['texts'] = df['texts'].apply(clean_text)

In [ ]:
df.tail()

,label,texts
11309,13,dn nyeda cnsvax uwec edu david nye dn neurolog...
11310,4,old mac k mac plus problem screen blank someti...
11311,3,installed dx cpu clone motherboard tried mount...
11312,1,require hyper sphere space point specifies sph...
11313,8,stolen pasadena pm blue white honda cbr rr cal...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x = df['texts']
y = df['label']

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vector = TfidfVectorizer(
    ngram_range=(1,2),
    min_df=2,
    max_df=0.85,
    max_features=20000
)

In [ ]:
x_train_vec = vector.fit_transform(x_train)
x_test_vec  = vector.transform(x_test)

In [ ]:
len(vector.get_feature_names_out())

20000

In [ ]:
import numpy as np

In [ ]:
y_train = np.array(y_train)

In [ ]:
y_test = np.array(y_test)

In [ ]:
y_test

array([ 1, 11, 15, ..., 14,  9, 14])

In [ ]:
y_train

array([10, 14, 10, ..., 18,  5, 15])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(x_train_vec , y_train)

RandomForestClassifier()

In [ ]:
y_pred = model.predict(x_test_vec)

In [ ]:
model.score(x_train_vec , y_train)

0.998295648221793

In [ ]:
model.score(x_test_vec , y_test)

0.6778736937755566

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(x_test_vec)

print("Train Accuracy:", model.score(x_train_vec, y_train))
print("Test Accuracy:", accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

Train Accuracy: 0.998295648221793
Test Accuracy: 0.6778736937755566
              precision    recall  f1-score   support

           0       0.70      0.48      0.57        92
           1       0.64      0.52      0.57       121
           2       0.61      0.77      0.68       116
           3       0.61      0.67      0.64       117
           4       0.73      0.63      0.68       110
           5       0.72      0.82      0.77       114
           6       0.68      0.71      0.69        97
           7       0.70      0.74      0.72       110
           8       0.73      0.71      0.72       133
           9       0.68      0.76      0.72       123
          10       0.80      0.84      0.82       126
          11       0.83      0.78      0.81       111
          12       0.59      0.66      0.62       107
          13       0.76      0.67      0.71       134
          14       0.69      0.77      0.73       108
          15       0.61      0.72      0.66       116
          16 

In [ ]:
import joblib

joblib.dump(model , 'DAY_8_NEWSPAPER_NLTK_model.pkl')

['DAY_8_NEWSPAPER_NLTK_model.pkl']

In [ ]:
vector

TfidfVectorizer(max_df=0.85, max_features=20000, min_df=2, ngram_range=(1, 2))

In [ ]:
joblib.dump(vector , 'day_8_NEWSPAPER_TFIDF.pkl')

['day_8_NEWSPAPER_TFIDF.pkl']

In [ ]:
df['texts'][0]

'wondering anyone could enlighten car saw day door sport car looked late early called bricklin door really small addition front bumper separate rest body know anyone tellme model name engine spec year production car made history whatever info funky looking car please e mail'

In [ ]:
df['label'][0]

np.int64(7)